# Milestone for project

This Jupyter notebook file contains Python code for cleaning and inserting data from the given csv file into a PostGreSQL database. The csv file in this case is the file containing 1mio. rows. The psycopg2 library is used to establish a connection to the DB and pandas is used as intermediary storage, as it contains a range of practical functions for working with data like this. In terms of tasks, this notebook contains the answer to Tasks 2, 3, and 4, all which can be found below with an explanation of the most important choices. As the overarching goal of the milestone assignment is to show a functional DB with support for the entire FakeNewsCorpus dataset (or at least the first million rows), efficiency both in loading and querying is of high importance. 

In [1]:
import pandas as pd 
import psycopg2, time, re

# Make connection to database
connection = psycopg2.connect(
    user = "athanar",
    host = "localhost",
    port = "5432",
    database = "datascience")
cursor = connection.cursor()

## Task 2

For task 2 a DB schema is necessary to properly store the data into the DB. Our choice of schema can be seen in the figure below, as well as the SQL code used to make it in the next cell. As the schema design has been discussed both at lectures, exercise classes as well as the discussion forums, this design should be similar to the commonly available DB schema.

![Schema](https://scontent-amt2-1.xx.fbcdn.net/v/t1.15752-9/93259206_2549607885294684_6675379213874233344_n.png?_nc_cat=101&_nc_sid=b96e70&_nc_ohc=02hwRasd1PAAX_2Me4X&_nc_ht=scontent-amt2-1.xx&oh=8a9252e5bb0c8b98c9d6e1583fb28b93&oe=5EC73563)

Central to the schema is the Article table. From this, reference tables, with many-to-many relations, tags and writtenby connect to related information. For tags the connected table Keyword contains all the unique meta keywords. Writtenby links articles to their authors. Typelinks functions as a many-to-one relation, where many articles can be of the same unique type in the Type table. Webpage contains the unique URL of an article and connects as a many-to-one relation to the domain of the article, as many articles and urls can come from the same domain. 

The use of intermediary tables webpage, typelinks, tags and writtenby is reasoned by higher efficiency. As unique IDs, often sequenced, are fast to compare and query, using such tables/relations is superior to querying/comparing the actual values. This is especially the case for the many-to-many relations. 

By only storing the unique domains, authors, types and keywords, redundant table entries are avoided. By only storing the actual relations in intermediary tables, the DB design becomes tighter. In addition the intermediary tables uses Foreign Keys on the linked IDs ensuring that each row contains a valid relation.

In [2]:
create_tables_str = """
    DROP SCHEMA public CASCADE;
    CREATE SCHEMA public;
    GRANT ALL ON SCHEMA public TO athanar, postgres;

    CREATE TABLE IF NOT EXISTS Article (
      articleID INT NOT NULL PRIMARY KEY,
      title VARCHAR NULL,
      content VARCHAR NULL,
      summary VARCHAR NULL,
      scrapedAt TIMESTAMP,
      insertedAt TIMESTAMP,
      updatedAt TIMESTAMP
    );

    CREATE TABLE IF NOT EXISTS Types (
      typeID SERIAL PRIMARY KEY,
      typeValue VARCHAR NOT NULL UNIQUE
    );

    CREATE TABLE IF NOT EXISTS Typelinks (
      articleID INT REFERENCES Article(articleID),
      typeID INT REFERENCES Types(typeID)
    );

    CREATE TABLE IF NOT EXISTS Keyword (
      keywordID SERIAL PRIMARY KEY,
      keywordValue VARCHAR NOT NULL UNIQUE
    );

    CREATE TABLE IF NOT EXISTS Tags (
      articleID INT REFERENCES Article(articleID),
      keywordID INT REFERENCES Keyword(keywordID)  
    );

    CREATE TABLE IF NOT EXISTS Domain (
      domainID SERIAL PRIMARY KEY,
      domainURL VARCHAR NOT NULL UNIQUE
    );

    CREATE TABLE IF NOT EXISTS Webpage (
      articleID INT REFERENCES Article(articleID),
      domainID INT REFERENCES Domain(domainID),
      webpageURL VARCHAR NOT NULL UNIQUE
    );

    CREATE TABLE IF NOT EXISTS Author (
      authorID SERIAL PRIMARY KEY,
      authorName VARCHAR NOT NULL UNIQUE
    );

    CREATE TABLE IF NOT EXISTS WrittenBy (
      articleID INT REFERENCES Article(articleID),
      authorID INT REFERENCES Author(authorID)
    );"""

In [3]:
# Read dataset
start_time = time.time()
reader = pd.read_csv(
    "1mio-raw.csv", 
    encoding='utf-8', 
    chunksize=10000)

# Read SQL file
def executeScriptFromFile(filename):
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()
    sqlCommands = sqlFile.split(';')
    for command in sqlCommands:
        try:
            cursor.execute(command)
        except:
            continue
            
def executeScriptFromString(string):
    sqlcommands = string.split(';')
    for command in sqlCommands:
        try:
            cursor.execute(command)
        except:
            continue
            
def clean(s): 
    if (isinstance(s, float)):
        return str(s) 
    # Replaces url's with < url >
    string = re.sub(r'http\S+', '< url >', s)
    
    # Replaces integers with <number>
    string = re.sub(r'\b\d+', ' <number> ', string)
    
    # Make lowercase and deletes newline
    string = s.lower().replace('\n\n', '')

    # Spaces between acceptable ASCII chars
    string = re.sub(r'([\x21-\x2f\x3a-\x60\x7b-\x7e])', r' \1', string)
    
    # Removes extended ASCII and unicode chars
    string = re.sub(r'[\u2000-\u2027\x80-\xff]', '', string)
    
    # Removes subsequent spaces
    string = re.sub(' +', ' ', string)
    
    string = string.strip()
    
    return string

### Continued Task 2

Above the script used to load the file can be seen. Pandas csv reader is used to read the file, which is done in chunks to improve performance (otherwise such a large file uses too much memory, especially when processed). Two functions are used to execute SQL commands, where the commands can either be passed as a file or as a string. The SQL string to make the actual tables in the schema can be seen in the cell above. 

A function for cleaning a string is also present in the cell above. This cleaning is primarily used on the title and content of article, to ensure readable information, as well as removing junk signs from the string. For some of the columns, such as author and keywords, cleaning is also done in the extractParts function below. The reason for this, is that the authors and keywords are stored in a long string and needs to be split into individual elements instead. While the information is already being processed it might as well be cleaned too.

Storing of the data into the PostGreSQL DB, makes use of the established connection and insert statements. The function insertTable below is used to mass insert data with the executemany statement, which is significantly faster than inserting individual elements into the table iteratively, as each INSERT statement comes with a large overhead. 

As the data from file has been read into a pandas dataframe, each colum can be chosen and operated on as seen in the loop, which iterates through all chunks. Each table is handled separately, though insertions happens for all tables in each chunk to ensure efficient insertion. 

First the data is cleaned and rows without an article ID are removed. Then they are inserted by order of relations. Article first as all other tables reference this, then types, then the links between and so on. In this way correct insertion is assured, as Foreign Key errors happen if a relation table is made before the actual id of the relations exist. 

To ensure uniqueness of all elements which should be unique, such as authors, keywords and domains, python sets are used. These allow for set operations, so no duplicates can exist and only new elements are inserted into tables. In addition dictionaries are used to maintain many-to-many relations such as authors and keywords, as the list of elements should be extracted from a string and paired with their relational articleID.

This process is timed to ensure proper running time, though it is still a slow process for such a large dataset. In addition, it is properly safeguarded with a message if an error occurs, so it can properly continue. This allows for targeted insertion afterwards, if a certain part fails, instead of having to redo the entire DB insertion. This function has successfully inserted the entire 1mio-rows dataset into a PostGreSQL DB.

Uncomment to run below, but beware that the SQL script will also drop DB schema as it is right now, so only run it once (necessary for testing). 

In [4]:
def storedata():
    # Variables:
    new_keywords = []
    new_tags = []
    new_domains = []
    new_authors = []    
    new_types = []
    
    # Inserts dataframe into database
    def insertTable(cols, vals, target):
        try: 
            sql = "INSERT INTO "+target+" (" +cols + ") VALUES (" + "%s,"*(len(vals.iloc[0])-1) + "%s)"
            cursor.executemany(sql, vals.values.tolist())
            connection.commit()
        except Exception as e:
            print("Something went wrong with: %s %s" % (target, str(e)))

    # Get typeid for type string
    def typeLookup(typeval):
        if (isinstance(typeval, float)):
            return 12
        else:
            return new_types.index(typeval) + 1

    # Extract comma separated parts of string column
    def extractParts(ids, column):
        tmp_dict = {}
        tmp = []
        for i in range(len(column)):
            if (isinstance(column.iloc[i], float)):
                tmp.extend(str(column.iloc[i]))
                tmp_dict[ids.iloc[i]] =  str(column.iloc[i])
            elif (column.iloc[i] == "[\'\']"):
                continue
            else:
                new_vals = (column.iloc[i]
                            .replace('[', '')
                            .replace(']', '')
                            .replace('\'', '')
                            .replace('\"', '')
                            .lower()
                            .split(', '))
                tmp.extend(new_vals)
                tmp_dict[ids.iloc[i]] = new_vals
        return set(tmp), tmp_dict
    
    i = 1
    for data in reader:
        # Size; Highly temporary for testing purposes. Can be adjusted to test smaller dataset
        if (i > 500):
            break
        try:
            # Clean data
            data['id'] = pd.to_numeric(data['id'], errors='coerce')
            data = data[data['id'].notna()]
            data['content'] = data['content'].apply(clean)
            data['title'] = data['title'].apply(clean)
            data['summary'] = data['summary'].apply(clean)
        except Exception as e:
            print("Cleaning went wrong in round: %s %s" % (i, str(e)))
        try:
            # Fetches article from dataframe
            article = data.iloc[:,[1,9,5,15,6,7,8]]
            articleval = "articleID, title, content, summary, scrapedAt, insertedAt, updatedAt"
            insertTable(articleval, article, "Article")
        except Exception as e:
            print("Article insertion went wrong in round: %s %s" % (i, str(e)))
            
        # Fetches types from dataframe, done like this as the first round 
        # finds all relevant types 
        if (len(new_types) < 1):
            types = data['type'].drop_duplicates().dropna()
            typeframe = pd.DataFrame(types).rename(columns={'type':'typeValue'})
            new_types = list(types)
            insertTable("typeValue", pd.DataFrame(types), "Types") 
            
        try:
            # Fills Typelinks
            articleid = data.iloc[:,[1]]
            typeid = data['type'].apply(typeLookup)
            typelinks = pd.concat([articleid, typeid], axis=1, ignore_index=True)
            insertTable("articleID, typeID", typelinks, "Typelinks")
        except Exception as e:
            print("Typelinks insertion went wrong in round:%s %s" % (i, str(e)))
        
        try: 
            # Fetches keywords from dataframe and inserts new keywords
            keywords, keyword_dict = extractParts(data['id'],data['meta_keywords'])
            keyword_list = list(keywords.difference(set(new_keywords)))
            new_keywords.extend(keyword_list)
            if (len(keyword_list) > 0):
                insertTable("keywordValue", pd.DataFrame(keyword_list), "Keyword")
        except Exception as e:
            print("Keywords insertion went wrong in round: %s %s" % (i, str(e)))
        
        try: 
            # Inserts keywordlinks
            tmp_kw = []
            for k, v in keyword_dict.items():
                for kword in v:
                    tmp_kw.append([k, new_keywords.index(kword)+1])
            insertTable("articleID, keywordID", pd.DataFrame(tmp_kw), "KeywordLinks")
        except Exception as e:
            print("Tags insertion went wrong in round: %s %s" % (i, str(e)))
            
        try: 
            # Fetches tags from dataframe and inserts new tags
            tags, tag_dict = extractParts(data['id'],data['tags'])
            tag_list = list(tags.difference(set(new_tags)))
            new_tags.extend(tag_list)
            if (len(tag_list) > 0):
                insertTable("tagValue", pd.DataFrame(tag_list), "Tag")
        except Exception as e:
            print("Keywords insertion went wrong in round: %s %s" % (i, str(e)))
        
        try: 
            # Inserts taglinks
            tmp_tag = []
            for t, v in tag_dict.items():
                for ts in v:
                    tmp_tag.append([t, new_tags.index(ts)+1])
            insertTable("articleID, tagID", pd.DataFrame(tmp_tag), "TagLinks")
        except Exception as e:
            print("Tags insertion went wrong in round: %s %s" % (i, str(e)))
        
        try:
            # Fetches domain from dataframe and inserts new domains
            domain = set(data.loc[:,'domain'])
            domain_list = list(domain.difference(set(new_domains)))
            new_domains.extend(domain_list)
            if (len(domain_list) > 0):
                insertTable("domainURL", pd.DataFrame(domain_list), "Domain")
        except Exception as e:
            print("Domain insertion went wrong in round: %s %s" % (i, str(e)))
        
        try:
            # Fetches webpageURL from dataframe and inserts
            new_webs = data['domain'].apply(lambda x: new_domains.index(x)+1)
            dom_frame = pd.DataFrame(
                {'id': data['id'], 'domain': new_webs, 'url': data['url']})
            insertTable("articleID, domainID, webpageurl", dom_frame, "Webpage")
        except Exception as e:
            print("Webpage insertion went wrong in round: %s %s" % (i, str(e)))
        
        try:
            # Fetches authors from dataframe and inserts new authors
            authors, authors_dict = extractParts(data['id'], data['authors'])
            author_list = list(authors.difference(set(new_authors)))
            new_authors.extend(author_list)
            if (len(author_list) > 0):
                insertTable("authorName", pd.DataFrame(author_list), "Author")
        except Exception as e:
            print("Authors insertion went wrong in round: %s %s" % (i, str(e)))
        
        try:
            # Inserts into writtenby
            tmp_aut = []
            for k, v in authors_dict.items():
                for kword in v:
                    tmp_aut.append([k, new_authors.index(kword)+1])
            insertTable("articleID, authorID", pd.DataFrame(tmp_aut), "WrittenBy")
        except Exception as e:
            print("Writtenby insertion went wrong in round: %s %s" % (i, str(e)))
        
        # Round counter for timing
        print("Round %d took %s seconds" % (i,time.time() - start_time))
        i = i+1

    print("Finished. Took %s seconds" % (time.time() - start_time))

In [5]:
executeScriptFromFile('create_tables.sql')
storedata()

Round 1 took 45.923566579818726 seconds
Round 2 took 87.02437114715576 seconds
Round 3 took 133.10276794433594 seconds
Round 4 took 236.31031560897827 seconds
Round 5 took 274.57006192207336 seconds
Round 6 took 313.32639956474304 seconds
Round 7 took 376.4915282726288 seconds
Round 8 took 474.39187836647034 seconds
Round 9 took 524.0554986000061 seconds
Round 10 took 571.4146285057068 seconds
Round 11 took 627.0398869514465 seconds
Round 12 took 676.7929012775421 seconds
Round 13 took 717.9069180488586 seconds
Round 14 took 823.6844954490662 seconds
Round 15 took 878.8675203323364 seconds
Round 16 took 961.156378030777 seconds
Round 17 took 1054.3395442962646 seconds
Round 18 took 1127.065241098404 seconds
Round 19 took 1230.3167827129364 seconds
Round 20 took 1265.990296125412 seconds
Round 21 took 1301.758017539978 seconds
Round 22 took 1339.954978942871 seconds
Round 23 took 1378.9899702072144 seconds
Round 24 took 1433.87468791008 seconds
Round 25 took 1535.4908590316772 seconds
R

## Task 3 and Task 4

Below are the functions used for Task 3 and Task 4, where the execute function are highly similar to the one used below, with prints and fetches as the difference. The queries themselves can be seen as well, where they are written as strings. Currently the exact same SQL commands are stored in a file, so they can be run at once and run in SQL command line directly too. But if there's an interest in running the functions with the string, the function and the strings are present.

The reliable domains are found by a waterfall of substrings, which significantly improves the efficiency relative to comparing the full tables to eachother. Prolific authors can be found in a simpler way where the elements are simply selected by crossreferencing the tables. While this could be improved with subqueries as well, we have found that it is almost instant as is, and thus no reason to significantly alter it.

The keywords are counted by making an inner join of a small part of the tags dataset on itself. It then compares all elements with all elements which is not the same as itself and gets the ones where the keyword is the same. Then the number of rows is counted, and divided by two, as the elements are paired both ways. As this is an expensive query, it is done on a small subset, but the number can be changed to any other number if there's an interest in seeing more common pairs.

For the exploration in task 4, four areas were examined. First we looked at domains and how many articles were written on each of each type. The output of this is interesting because it gives insight into which types are most frequently written, where the quality as expected is classified as low (Types unreliable, fake, political and conspiracy). 

Next the amount of articles written of each type is counted, which provides a simple overview of the distribution of articles. Then the amount of articles written by each author is found and finally the amount of articles of each type is found. 

In [6]:
def queryfile(filename):
    # Read SQL file
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()
    sqlCommands = sqlFile.split(';')
    for command in sqlCommands:
        try:
            cursor.execute(command)
            print("\n-----------------------------------Query---------------------------------------\n")
            print(pd.DataFrame(cursor.fetchall()))
        except:
            continue 
def querystr(string):
    sqlCommands = string.split(';')
    for command in sqlCommands:
        try:
            cursor.execute(string)
            print("\n-----------------------------------Query---------------------------------------\n")
            print(pd.DataFrame(cursor.fetchall()))
        except:
            continue

### Task 3 scripts

In [7]:
# Reliable domains of news articles for Task 3.1
reliable_domains = """
    SELECT domainURL
    FROM Domain
    WHERE domainID in (
        SELECT domainID
        FROM Webpage
        Where articleID in (
            SELECT articleID 
            FROM typelinks
            WHERE typeID = 8 
            AND articleID in (
                SELECT articleID
                FROM Article
                WHERE scrapedAt >= to_timestamp(2018-1-15)
            )
        )
    );"""

 
# Most prolific authors of fake type news articles for Task 3.2
prolific_fake_authors = """
    SELECT authorName, COUNT(w.authorid) 
    FROM writtenby w, typelinks t, author a 
    WHERE w.articleid = t.articleid 
        AND t.typeid = 7 
        AND w.authorID = a.authorID
    GROUP BY a.authorName, w.authorid 
    ORDER BY count DESC
    LIMIT 20; """

# Pairs of article ids with the exact same set of keywords for Task 3.3
count_keywordpairs = """
    WITH tags_small AS (SELECT * from tags where articleID <= 500),
         article_small AS (SELECT DISTINCT articleID FROM tags_small)
    SELECT COUNT(DISTINCT t1.articleID) / 2
    FROM tags_small t1
    INNER JOIN tags_small t2 
    ON t1.articleID <> t2.articleID AND t1.keywordID = t2.keywordID;"""

### Task 4 scripts

In [8]:
hate_articles_domain = """
    SELECT domainURL, typeValue, COUNT(w.domainID)
    FROM Webpage w, Domain d, typelinks tl, Types t
    WHERE w.domainID in (
        SELECT domainID
        FROM Webpage
        Where articleID in (
            SELECT articleID 
            FROM typelinks
            WHERE typeID = 2 
            )
        ) 
    AND w.domainID = d.domainID
    AND w.articleID = tl.articleID
    AND tl.typeID = t.typeID
    GROUP BY d.domainURL, typeValue
    ORDER BY count DESC;"""

articles_per_type = """
    SELECT typeValue, COUNT(articleID)
    FROM Types t, typelinks tl
    WHERE t.typeID = tl.typeID
    GROUP BY typeValue
    ORDER BY count DESC;"""

articles_per_author = """
    SELECT authorName, COUNT(w.articleID)
    FROM Writtenby w, Author a 
    WHERE w.authorID = a.authorID
    GROUP BY authorName
    ORDER BY count DESC
    LIMIT 100;"""

article_type_count_author = """
    SELECT authorName, typeValue, COUNT(a.authorID)
    FROM Author a, Writtenby w, typelinks tl, Types t
    WHERE w.articleID = tl.articleID AND a.authorID = w.authorID AND tl.typeID = t.typeID
    GROUP BY authorName, typeValue
    ORDER BY count DESC
    LIMIT 100;"""

In [9]:
#queryfile('queries.sql')
#queryfile('queries2.sql')

# Task 5
The webscraping algorithm has been created with the help of pandas, beautifulsoup and urllib. Instead of starting the iteration from A, the specific input has been set to start iterating from articles beginning with J. This has been done to dramatically decrease running time. The idea is to start the iteration from an arbitrary URL and then search through the headers of the HTML code to see whether or not the desired articles have been found. Although this specific code handles the URL of articles starting with J, slight modifications could have been made for the algorithm to start all the way from A-Z and only selecting articles in question. It then creates a list of article URL's   where url requests are made and information of importance is fetched and stored in a pandas dataframe.

A simple text cleaning has also been implemented for the only purpose to clean the html code - discarding all unwanted data. 
The pandas dataframe returned by the algorithm consists of 3 columns; title, date and content. The title and date has specifically been found using the functionality of the beautifulsoup package, which makes it possible to iterate through the HTML code more freely. Out of the approximately 7500 articles present in Wikinews around 3000 (from the range J-T) is stored and returned in the pandas dataframe.
One of the challenging asepcts of doing this exercise was to get an understanding of the HTML code of a website, what we wanted to extract, and what felt less important. This felt very different from what we are used to seeing, but nevertheless a heirarcically structure is present and ultimately this helped us to get a better understanding of how webpages are constructed.

In [10]:
import re

def webscraping_cleanup(string):
    retval = re.sub(r'<.+?>', '', string)
    retval = re.sub(r'\b.+\xa0', '', retval).replace('\n, ', '').lower()
    retval = re.sub(r'\n\n\n\n\n.+', '', retval).replace('\n','')
    return retval

In [11]:
import bs4 as bs
import urllib.request
import pandas as pd

# Start letters of articles to find: [J-T]

def webscraping(first_article, end_article, URL):
    column_names = ['title', 'date', 'content']
    df = pd.DataFrame(columns=column_names)

    condition = True
    while condition:
        source = urllib.request.urlopen("https://en.wikinews.org" + URL).read()
        soup = bs.BeautifulSoup(source, 'lxml')

        all_articles = soup.find_all('div', attrs={'id':'mw-pages'})

        next_list = []
        a_list = []
        headers = []
        for a in all_articles:
            # Finds html code for next and previous page (although only next is used)    
            next_list.extend(a.find_all(title='Category:Politics and conflicts'))
            # Finds information about all articles starting with [J-T]
            a_list.extend(a.find_all(title=re.compile(r'^['+first_article+'-'+end_article+']\w')))
            # Finds headers of all_articles
            headers.extend(a.find_all(re.compile(r'^h[1-6]$')))
        
        # Find last element of headers
        if headers[-1].text >= first_article and headers[-1].text <= end_article:
            condition = True
            print('Processing articles: ',headers[-1].text)
        else: 
            condition = False

        # Finds URL of next source to be iterated through    
        URL = next_list[-1]['href']
        for url in a_list:
            # Makes urlrequest to all articles found in the desired range: [J-T]
            article_source = urllib.request.urlopen("https://en.wikinews.org" + url['href']).read()
            article_soup = bs.BeautifulSoup(article_source, 'lxml')
    
            article_title = article_soup.head.title.text.replace(' - Wikinews, the free news source', '')
    
            # Some publishDates appear as None in html
            try:
                article_date = article_soup.find(id='publishDate')['title']
            except:
                article_date = 'unknown'
            
            article_text = article_soup.find_all('div', attrs={'class':'mw-parser-output'})
            p_list = []
            for p in article_text:
                #TODO: Find text from article other than that located in <p>
                p_list.extend(p.find_all('p'))
    
            # Simple text cleanup
            article_text = webscraping_cleanup(str(p_list))
            
            df = df.append({'title':article_title, 'date':article_date, 'content':article_text}, ignore_index=True)
        
    return df

In [12]:
#df = webscraping('J', 'T', '/w/index.php?title=Category:Politics_and_conflicts&pageuntil=Jesse+Jackson+apologizes+for+comment+about+Barack+Obama#mw-pages')
#df